普通Linux系统下通过查看/proc/ioports文件可以得到相关控制器或设置使用的I/O地址范围
32kB-1KB=0x7c00(111 1100 0000 0000)

上电后，内存从主板rom中加载bios程序到固定位置，并自动执行；
bios程序加载中断向量表、硬盘参数等，将CPU初始化到可用状态。
将CPU cs寄存器地址为0xF000， 基段地址为0xFFFF0000，IP值为0xFFF0，所以此时的代码地址是0xFFFF0000+0xFFF0=0xFFFFFFF0 。
BIOS提前在内存0xFFFFFFF0出存放一条JMP指令，跳转到内存中bios程序段的一个固定地址；
bios此处代码会经过一系列硬件检测和初始化操作后，会将硬盘引导扇区的512B数据导入到0x7c00处。
引导扇区中的代码把硬盘的分区表信息载入到内存，查找标记为活动的分区，并在该分区中加载执行grub引导程序的代码；
grub启动后，加载文件系统，找到配置文件grub.cfg; grub.cfg指出了内核的在文件中的位置